In [83]:
!pip install torch
!pip install transformers
!pip install textstat
!pip install scikit_learn
!pip install datasets

In [84]:
import pandas as pd
import sklearn
import os
import random
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" # needed for making torch work?
import torch
from transformers import DistilBertTokenizer, DistilBertModel

In [85]:
# Read authentication keys from environmental variables
_open_ai_tkn = os.environ.get('OPENAI_KEY')
_project_tkn = os.environ.get('OPENAI_PROJECT')
_organisation_tkn = os.environ.get('OPENAI_ORG')

In [86]:
from datasets import load_dataset

# Load the dataset splits
dataset = load_dataset("Annanay/aml_song_lyrics_balanced")
train_df = dataset['train'].to_pandas()  # Convert train split to Pandas DataFrame
test_df = dataset['test'].to_pandas()    # Convert test split to Pandas DataFrame


In [87]:
from openai import OpenAI

client = OpenAI(
  organization=_organisation_tkn,
  project=_project_tkn,
  api_key=_open_ai_tkn
)

In [88]:
def prompt_song(test_song):
    test_song_lines = test_song.split('\n')
    test_song_lines = [s for s in test_song_lines if len(s) > 0]
    old_lines = []
    new_lines = []
    for i in range(0, len(test_song_lines) - 7):
        question = "Please write a unique one-line lyric between these two stanzas of three lines: \n"
        group_1 = '\n'.join(test_song_lines[i:i+3])
        group_2 = '\n'.join(test_song_lines[i+4: i+7])
        prompt = f"{question}{group_1}\n\n{group_2}\n"
        #print(prompt)
        response = client.chat.completions.create(
            messages=[{
                "role": "user",
                "content": prompt
            }],
            model="gpt-4o-mini",
            max_completion_tokens=100
        )
        new_line = response.choices[0].message.content.split('\n')[0]
        old_lines.append(test_song_lines[i+3])
        new_lines.append(new_line)
    return new_lines, old_lines

In [89]:
import random
import pronouncing

# Extract rhyming words
def extract_rhyme(word):
    """Extracts rhyming words for the given word."""
    if not word:
        return set()
    rhymes = pronouncing.rhymes(word)
    return set(rhymes) if rhymes else set()

# Check rhyme scheme
def check_rhyme_scheme(new_line, reference_lines):
    """
    Checks if the new line fits the rhyme scheme of the reference lines.
    Returns True if it matches, False otherwise.
    """
    try:
        # Get the last word of the new line
        new_rhyme = extract_rhyme(new_line.split()[-1])
        
        # Get rhymes for the last words of the reference lines
        reference_rhymes = [
            extract_rhyme(line.split()[-1]) for line in reference_lines
        ]
        
        # Check if the new line matches any rhyme in the reference stanza
        for ref_rhyme in reference_rhymes:
            if new_rhyme & ref_rhyme:
                return True  # Rhyme matches
        
        return False  # No rhyme match
    except IndexError:
        return False  # Handles cases where lines are empty or malformed

# Prompt the song
def prompt_song(test_song, client):
    """
    Randomly selects a stanza from the song and prompts ChatGPT to generate a one-line lyric.
    """
    # Split and clean the song into lines
    test_song_lines = [line.strip() for line in test_song.split('\n') if line.strip()]
    
    # Randomly select a valid index for the target line
    i = random.randint(3, len(test_song_lines) - 4)
    
    # Extract context
    group_1 = '\n'.join(test_song_lines[i - 3:i])
    group_2 = '\n'.join(test_song_lines[i + 1:i + 4])
    expected_line = test_song_lines[i]

    # Create the prompt
    question = "Please write a unique one-line lyric between these two stanzas of three lines: \n"
    prompt = f"{question}{group_1}\n\n{group_2}\n"

    # Generate response using ChatGPT
    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt
        }],
        model="gpt-4o-mini",
        max_tokens=100
    )
    new_line = response.choices[0].message.content.split('\n')[0].strip()

    print(f"\nPrompt:\n{prompt}")
    print(f"Generated Line: {new_line}")
    print(f"Expected Line: {expected_line}")

    # Return the generated line, expected line, and reference stanza
    return new_line, expected_line, group_1.split('\n') + group_2.split('\n')


In [89]:
import random
import pronouncing

# Extract rhyming words
def extract_rhyme(word):
    """Extracts rhyming words for the given word."""
    if not word:
        return set()
    rhymes = pronouncing.rhymes(word)
    return set(rhymes) if rhymes else set()

# Check rhyme scheme
def check_rhyme_scheme(new_line, reference_lines):
    """
    Checks if the new line fits the rhyme scheme of the reference lines.
    Returns True if it matches, False otherwise.
    """
    try:
        # Get the last word of the new line
        new_rhyme = extract_rhyme(new_line.split()[-1])
        
        # Get rhymes for the last words of the reference lines
        reference_rhymes = [
            extract_rhyme(line.split()[-1]) for line in reference_lines
        ]
        
        # Check if the new line matches any rhyme in the reference stanza
        for ref_rhyme in reference_rhymes:
            if new_rhyme & ref_rhyme:
                return True  # Rhyme matches
        
        return False  # No rhyme match
    except IndexError:
        return False  # Handles cases where lines are empty or malformed

# Prompt the song
def prompt_song(test_song, client):
    """
    Randomly selects a stanza from the song and prompts ChatGPT to generate a one-line lyric.
    """
    # Split and clean the song into lines
    test_song_lines = [line.strip() for line in test_song.split('\n') if line.strip()]
    
    # Randomly select a valid index for the target line
    i = random.randint(3, len(test_song_lines) - 4)
    
    # Extract context
    group_1 = '\n'.join(test_song_lines[i - 3:i])
    group_2 = '\n'.join(test_song_lines[i + 1:i + 4])
    expected_line = test_song_lines[i]

    # Create the prompt
    question = "Please write a unique one-line lyric between these two stanzas of three lines: \n"
    prompt = f"{question}{group_1}\n\n{group_2}\n"

    # Generate response using ChatGPT
    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt
        }],
        model="gpt-4o-mini",
        max_tokens=100
    )
    new_line = response.choices[0].message.content.split('\n')[0].strip()

    print(f"\nPrompt:\n{prompt}")
    print(f"Generated Line: {new_line}")
    print(f"Expected Line: {expected_line}")

    # Return the generated line, expected line, and reference stanza
    return new_line, expected_line, group_1.split('\n') + group_2.split('\n')


In [90]:
# Define the test song
test_song1 = 'In this green and pleasant land\nWe have a dream to understand\nIn the mountains of the mind\nThere is a spirit you will find\nJust like the angel from above\nSent to deliver words of love\nAncient cross and Zion star\nEastern ways and praise to Jah\n\nThis is our land\nThis is your land\nThis is our inheritance\nTo lead you on a merry dance\nIn the beginning there was light\nShining path and journeys bright\nWhen the nations pray as one\nThen will the races all be won\nIn this green and pleasant land\nWe have a dream to understand\nOpen paradises gate\nDance on the land and celebrate\n\nSay no matter what your colour\nYour race or your culture\nThis is our inheritance\nTo lead you on a merry dance'
test_song2 = "Love and happiness...\nSomething that can make you do wrong, make you do right...\nLove...\n\nLove and happiness\nWait a minute...\nSomething's going wrong\nSomeone's on the phone\nThree o'clock in the morning\nTalkin' about how she can make it right\nWell\nHappiness is when you really feel good with somebody\nNothing wrong with being in one with someone\nOh, baby, love and happiness\nLove and happiness...\nLove and happiness...\nLove and happiness\nYou be good to me\nI'll be good to you\nWe'll be together\nWe'll see each other\nWalk away with victory\nOh baby\nLove and happiness...\nLove and happiness...\n\nMake you do right... love'll make you do wrong...\nMake you come home early...\nMake you stay out all night long...\nThe power of love...\n\nWait a minute\nLet me tell you...\nThe power of love...\nMake you do right... love'll make you do wrong...\nMake you want to dance...\nLove and happiness...\nLove and happiness...\n\nLove is... wait a minute... love is...\nWalkin' together...\nTalkin' together...\nSay it again...\nSay it together...\nMmmm...."

# Generate one line and check rhyme scheme
new_line, expected_line, reference_lines = prompt_song(test_song1, client)
fits_rhyme = check_rhyme_scheme(new_line, reference_lines)
# Output the result
print(f"\nDoes the generated line fit the rhyme scheme? {fits_rhyme}")

# Generate one line and check rhyme scheme
new_line, expected_line, reference_lines = prompt_song(test_song2, client)
fits_rhyme = check_rhyme_scheme(new_line, reference_lines)

# Output the result
print(f"\nDoes the generated line fit the rhyme scheme? {fits_rhyme}")




Prompt:
Please write a unique one-line lyric between these two stanzas of three lines: 
Just like the angel from above
Sent to deliver words of love
Ancient cross and Zion star

This is our land
This is your land
This is our inheritance

Generated Line: Guided by faith, we'll raise our hearts anew.
Expected Line: Eastern ways and praise to Jah

Does the generated line fit the rhyme scheme? False

Prompt:
Please write a unique one-line lyric between these two stanzas of three lines: 
Walk away with victory
Oh baby
Love and happiness...

Make you do right... love'll make you do wrong...
Make you come home early...
Make you stay out all night long...

Generated Line: In the dance of desire, we’re spinning on the edge...
Expected Line: Love and happiness...

Does the generated line fit the rhyme scheme? False
